In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow import keras
from numpy.random import seed
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import lightgbm as gbm
from torch.utils.data import Dataset, DataLoader
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import KFold

# Dataloading Functions

In [3]:
# Loads the Cover Type dataset while dropping rows with NaN
def load_cover_type():
    cov_train_df = pd.read_csv('covtype_train.csv')
    cov_train_df = cov_train_df[cov_train_df['Cover_Type'].notna()]
    
    X_train_cov = cov_train_df.drop(['Cover_Type'], axis=1)
    y_train_cov = cov_train_df['Cover_Type']

    return cov_train_df, X_train_cov, y_train_cov

In [4]:
# Loads the HELOC dataset while dropping rows with NaN and negative values
def load_heloc():
    heloc_train_df = pd.read_csv('heloc_train.csv')

    heloc_train_df[heloc_train_df.drop('RiskPerformance', axis=1) <0 ] = np.nan
    heloc_train_df = heloc_train_df.dropna()
    
    heloc_train_df = heloc_train_df[heloc_train_df['RiskPerformance'].notna()]
    
    X_heloc = heloc_train_df.drop('RiskPerformance', axis=1)
    y_heloc = heloc_train_df['RiskPerformance']

    return heloc_train_df, X_heloc, y_heloc

In [5]:
# Loads the HIGGS dataset while dropping rows with NaN values, -999 values, correlated columns: 
    #PRI_jet_all_pt
    #PRI_met_sumet
    #DER_mass_vis
#as well as the Weight column
def load_higgs(corr=True):
    higgs_train_df = pd.read_csv('higgs_train.csv')
    higgs_train_df = higgs_train_df[higgs_train_df['Label'].notna()]
    higgs_train_df = higgs_train_df.drop('EventId', axis=1)
    higgs_train_df = higgs_train_df.replace(-999.0, np.NaN)
    higgs_train_df = higgs_train_df.dropna()

    if corr:
        higgs_train_df = higgs_train_df.drop(['PRI_jet_all_pt', "PRI_met_sumet", 'DER_mass_vis'], axis=1)
        
    X_higgs = higgs_train_df.drop('Label', axis=1)
    X_higgs = X_higgs.drop('Weight', axis=1)
    y_higgs = higgs_train_df['Label']

    return higgs_train_df, X_higgs, y_higgs

In [49]:
#Adds synthetic data to the Cover Type dataset to compensate for underepresented classes using SMOTE
def resample_cover_type(X_train_cov, y_train_cov):
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_cov, y_train_cov)

    return X_resampled, y_resampled

In [7]:
#Encodes the labels of each dataset and splits them into a training and test subset
def le_split(X, y, t_size=0.3):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = t_size, random_state=42)

    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)

    return X_train, X_test, y_train, y_test

In [8]:
# Performs grid search to find optimal hyperparamteres for a given model
def train_model(model, X, y, grid=False, params={} ):
    if grid:
        grid_search = GridSearchCV(model, params, scoring="accuracy", n_jobs=-1, cv=10, verbose = 6)
        results = grid_search.fit(X, y)
        model = results.best_estimator_
        print(results.best_params_)
    model.fit(X, y)

    return model

In [9]:
# Evaluates a given model on the test subset
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    
    acc = accuracy_score(y, predictions)
    
    print("Accuracy:", acc)
    print("\nClassification Report:")
    print(classification_report(y, predictions))

    return predictions

In [61]:
# Initialize and run the TabNet model for a given dataset
def run_tabnet(X, y):

    X= X.to_numpy()
    y= y.to_numpy()

    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    CV_score_array    = []


    counter = 0
    
    for train_index, test_index in kf.split(X):
        if counter == 1:
            break
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]

        tb_cls = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-3),
                       scheduler_params={"step_size":10, "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                        mask_type = 'entmax', 
                        device_name = 'cpu'
                       )
        
        tb_cls.fit(X_train,y_train,
               eval_set=[(X_train, y_train), (X_valid, y_valid)],
               eval_name=['train', 'valid'],
               eval_metric=['accuracy'],
               max_epochs=1000 , patience=10,
               batch_size=28, drop_last=True)            
        CV_score_array.append(tb_cls.best_cost)
        counter += 1

    print("Total Accuracy: ", sum(CV_score_array) / len(CV_score_array))

    return tb_cls

In [13]:
#Loads the final submissions datasets
def load_tests():
    y_final_cov = pd.read_csv('covtype_test.csv')
    y_final_heloc = pd.read_csv('heloc_test.csv')
    y_final_higgs = pd.read_csv('higgs_test.csv').drop(['EventId', 'Weight', 'PRI_jet_all_pt', "PRI_met_sumet", 'DER_mass_vis' ], axis=1)

    return y_final_cov, y_final_heloc, y_final_higgs

In [14]:
#Creates the concatenated final predictions as well as the predictions for each model separately
def get_final_predictions(model_cov, model_heloc, model_higgs, isTab=False):
    y_final_cov, y_final_heloc, y_final_higgs = load_tests()

    if isTab:
        final_preds_cov = model_cov.predict(y_final_cov.to_numpy())
        final_preds_heloc = model_heloc.predict(y_final_heloc.to_numpy())
        final_preds_higgs = model_higgs.predict(y_final_higgs.to_numpy())
    else:
        final_preds_cov = model_cov.predict(y_final_cov)+1
        final_preds_heloc = model_heloc.predict(y_final_heloc)
        final_preds_higgs = model_higgs.predict(y_final_higgs)

    final_preds = np.concatenate((final_preds_cov, final_preds_heloc, final_preds_higgs))

    return final_preds, final_preds_cov, final_preds_heloc, final_preds_higgs

# Datasets Loading

## Load Cover Type

In [15]:
cov_train_df, X_cov, y_cov = load_cover_type()

In [16]:
cov_train_df['Cover_Type'].value_counts()

2    28248
1    21297
3     3607
7     2052
6     1706
5      932
4      259
Name: Cover_Type, dtype: int64

In [17]:
X_resampled, y_resampled = resample_cover_type(X_cov, y_cov)

In [18]:
pd.DataFrame(y_resampled).value_counts()

Cover_Type
1             28248
2             28248
3             28248
4             28248
5             28248
6             28248
7             28248
dtype: int64

In [19]:
X_train_cov, X_test_cov, y_train_cov, y_test_cov = le_split(X_resampled, y_resampled)

## Load Heloc

In [20]:
heloc_train_df, X_heloc, y_heloc = load_heloc()

In [21]:
X_train_heloc, X_test_heloc, y_train_heloc, y_test_heloc = le_split(X_heloc, y_heloc, t_size =0.2)

## Load Higgs

In [22]:
higgs_train_df, X_higgs, y_higgs = load_higgs()

In [23]:
#Correlation for Higgs Dataset
corr = higgs_train_df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\johnp\AppData\Local\Temp\ipykernel_13256\3153411704.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = higgs_train_df.corr()
C:\Users\johnp\AppData\Local\Temp\ipykernel_13256\3153411704.py:3: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm').set_precision(2)


,DER_mass_MMC,DER_mass_transverse_met_lep,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,Weight
DER_mass_MMC,1.00,0.25,0.02,-0.03,-0.02,0.02,0.52,0.02,0.09,0.08,-0.02,-0.04,0.18,-0.01,-0.01,0.26,0.01,0.00,0.09,0.00,0.01,-0.01,0.00,-0.00,-0.01,0.01,-0.00,0.17
DER_mass_transverse_met_lep,0.25,1.00,-0.18,-0.13,-0.14,0.12,0.19,0.10,-0.01,0.27,-0.34,-0.11,-0.07,0.00,-0.00,0.28,0.00,0.01,-0.06,-0.01,0.05,-0.11,0.00,0.00,-0.03,0.00,0.01,0.35
DER_pt_h,0.02,-0.18,1.00,-0.04,0.19,-0.02,-0.67,0.16,0.72,0.10,0.36,0.01,0.45,-0.01,0.01,0.40,-0.01,-0.01,0.78,0.01,0.14,0.75,0.00,0.00,0.36,-0.00,-0.01,-0.20
DER_deltaeta_jet_jet,-0.03,-0.13,-0.04,1.00,0.80,-0.84,0.04,-0.18,-0.11,-0.08,0.10,0.64,0.01,-0.00,-0.00,-0.08,-0.01,-0.01,-0.05,-0.01,-0.18,-0.05,-0.00,0.00,-0.09,-0.01,0.00,-0.29
DER_mass_jet_jet,-0.02,-0.14,0.19,0.80,1.00,-0.76,-0.12,-0.07,0.23,-0.05,0.15,0.48,0.11,-0.01,0.00,0.02,-0.00,-0.01,0.14,-0.01,-0.08,0.28,-0.00,0.00,0.23,-0.01,-0.00,-0.28
DER_prodeta_jet_jet,0.02,0.12,-0.02,-0.84,-0.76,1.00,0.01,0.13,0.02,0.06,-0.10,-0.56,-0.04,0.01,0.00,0.04,0.00,0.01,0.00,0.01,0.12,-0.02,-0.00,-0.01,0.01,0.00,0.00,0.25
DER_deltar_tau_lep,0.52,0.19,-0.67,0.04,-0.12,0.01,1.00,-0.11,-0.45,0.09,-0.25,-0.01,-0.32,0.00,-0.02,-0.12,0.01,0.01,-0.41,-0.00,-0.11,-0.48,0.01,0.00,-0.24,0.01,0.01,0.23
DER_pt_tot,0.02,0.10,0.16,-0.18,-0.07,0.13,-0.11,1.00,0.42,0.05,-0.04,-0.15,0.05,-0.00,0.01,0.10,-0.00,-0.01,0.15,0.00,0.60,0.19,-0.01,0.00,0.33,-0.00,-0.00,-0.03
DER_sum_pt,0.09,-0.01,0.72,-0.11,0.23,0.02,-0.45,0.42,1.00,0.13,0.14,-0.04,0.46,-0.01,0.01,0.45,-0.00,-0.00,0.49,0.00,0.40,0.88,-0.00,0.00,0.73,-0.00,-0.01,-0.14
DER_pt_ratio_lep_tau,0.08,0.27,0.10,-0.08,-0.05,0.06,0.09,0.05,0.13,1.00,-0.08,-0.06,-0.47,-0.00,0.00,0.75,0.00,-0.00,0.00,0.00,0.06,0.07,0.00,-0.00,0.05,0.00,0.00,0.24


In [24]:
X_train_higgs, X_test_higgs, y_train_higgs, y_test_higgs = le_split(X_higgs, y_higgs)

# XGBoost

## Cover Type

---

In [64]:
n_estimators = range(50, 400, 50)
max_depth = range(1, 11, 2)
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]

param_grid_cov = dict(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)

model_cov = xgb.XGBClassifier(n_estimators = 200, learning_rate = 0.1, nthread=-1)

model_cov = train_model(model_cov, X_train_cov, y_train_cov, grid=True, params= param_grid_cov)

### Run this if best parameters are known

In [52]:
model_cov = xgb.XGBClassifier(n_estimators = 350, learning_rate = 0.3, max_depth = 9, nthread=-1) #best parameters shown

model_cov = train_model(model_cov, X_train_cov, y_train_cov)

## Evaluate Cover Type Model

In [30]:
preds_cov = evaluate_model(model_cov, X_test_cov, y_test_cov)

Accuracy: 0.9760624399453819

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      8410
           1       0.93      0.92      0.92      8476
           2       0.99      0.99      0.99      8399
           3       1.00      1.00      1.00      8414
           4       0.99      1.00      0.99      8463
           5       0.99      1.00      0.99      8640
           6       0.99      1.00      1.00      8519

    accuracy                           0.98     59321
   macro avg       0.98      0.98      0.98     59321
weighted avg       0.98      0.98      0.98     59321



## Heloc

---

### The following code adds some generated data to the HELOC dataset 

In [42]:
#Adds synthetic data to the HELOC dataset to compensate for the small training dataset
oversample = SMOTE()
X_train_heloc, y_train_heloc = oversample.fit_resample(X_train_heloc, y_train_heloc)

In [133]:
n_estimators = range(50, 400, 50)
max_depth = range(1, 11, 2)
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]

param_grid_heloc = dict(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)

model_heloc = xgb.XGBClassifier()

model_heloc = train_model(model_heloc, X_train_heloc, y_train_heloc, grid=True, params= param_grid_heloc)

Fitting 10 folds for each of 210 candidates, totalling 2100 fits
{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 350}


### Run this if the best parameters are known

In [53]:
model_heloc = xgb.XGBClassifier(learning_rate = 0.1, max_depth= 1, n_estimators= 350)

model_heloc = train_model(model_heloc, X_train_heloc, y_train_heloc)

## Evaluate the HELOC model

In [44]:
preds_heloc = evaluate_model(model_heloc, X_test_heloc, y_test_heloc)

Accuracy: 0.6914414414414415

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.74      0.75       285
           1       0.56      0.61      0.59       159

    accuracy                           0.69       444
   macro avg       0.67      0.67      0.67       444
weighted avg       0.70      0.69      0.69       444



## Higgs

---

In [127]:
model_higgs = xgb.XGBClassifier(nthread=-1)

n_estimators = range(50, 400, 50) # tune number of trees
max_depth = range(1, 11, 2) # tune size of decision trees
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3] # tune the learning rate

param_grid_higgs = dict(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)

grid_search_higgs = GridSearchCV(model_higgs, param_grid_higgs, scoring="accuracy", n_jobs=-1, cv=5, verbose=5)
grid_search_higgs.fit(X_train_higgs, y_train_higgs)

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, nthread=-1,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
                         'max_depth': range(1, 11, 2),
                         'n_estimators': range(50, 400, 50)},
             scoring='accuracy', verbose=5)

In [128]:
grid_search_higgs.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 250}

In [54]:
#model_higgs = grid_search_higgs.best_estimator_

#model_higgs = xgb.XGBClassifier(eta = 0.2, max_depth = 6, gamma = 5)
model_higgs = xgb.XGBClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 250)
model_higgs.fit(X_train_higgs, y_train_higgs)

predictions_higgs = model_higgs.predict(X_test_higgs)

#Calculating accuracy
accuracy_higgs = accuracy_score(y_test_higgs, predictions_higgs)

print("Accuracy:", accuracy_higgs)
print("\nClassification Report:")
print(classification_report(y_test_higgs, predictions_higgs))


Accuracy: 0.8432330564668109

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      7679
           1       0.83      0.83      0.83      6648

    accuracy                           0.84     14327
   macro avg       0.84      0.84      0.84     14327
weighted avg       0.84      0.84      0.84     14327



In [45]:
y_final_cov = pd.read_csv('covtype_test.csv')
y_final_heloc = pd.read_csv('heloc_test.csv')
y_final_higgs = pd.read_csv('higgs_test.csv').drop(['EventId', 'Weight', 'PRI_jet_all_pt', "PRI_met_sumet", 'DER_mass_vis' ], axis=1)

final_preds_cov = model_cov.predict(y_final_cov)+1
final_preds_heloc = model_heloc.predict(y_final_heloc)
final_preds_higgs = model_higgs.predict(y_final_higgs)

In [41]:
y_final_cov = pd.read_csv('covtype_test.csv')

In [46]:
#model_cov.predict(y_final_cov.to_numpy())
final_preds_cov = model_cov.predict(y_final_cov.to_numpy())

# TabNet

## Cover Type

---

In [62]:
model_cov = run_tabnet(X_cov, y_cov)

# save tabnet model
saving_path_name = "./tabnet_model_cov"
saved_filepath = model_cov.save_model(saving_path_name)

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.35809 | train_accuracy: 0.66816 | valid_accuracy: 0.66982 |  0:00:25s
epoch 1  | loss: 0.80334 | train_accuracy: 0.69682 | valid_accuracy: 0.69934 |  0:00:49s
epoch 2  | loss: 0.74155 | train_accuracy: 0.71568 | valid_accuracy: 0.7156  |  0:01:13s
epoch 3  | loss: 0.71294 | train_accuracy: 0.71779 | valid_accuracy: 0.72197 |  0:01:38s
epoch 4  | loss: 0.69376 | train_accuracy: 0.72141 | valid_accuracy: 0.72283 |  0:02:03s
epoch 5  | loss: 0.67787 | train_accuracy: 0.72851 | valid_accuracy: 0.72911 |  0:02:27s
epoch 6  | loss: 0.66957 | train_accuracy: 0.73221 | valid_accuracy: 0.73298 |  0:02:51s
epoch 7  | loss: 0.66367 | train_accuracy: 0.73511 | valid_accuracy: 0.73505 |  0:03:16s
epoch 8  | loss: 0.65089 | train_accuracy: 0.7386  | valid_accuracy: 0.73918 |  0:03:40s
epoch 9  | loss: 0.64806 | train_accuracy: 0.74034 | valid_accuracy: 0.73935 |  0:04:04s
epoch 10 | loss: 0.64041 | train_accuracy: 0.73929 | valid_accuracy: 0.74056 |  0:04:28s
epoch 11 | loss: 0.63

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Total Accuracy:  0.7957146545047759
Successfully saved model at ./tabnet_model_cov.zip


## HELOC

---

In [57]:
model_heloc = run_tabnet(X_heloc, y_heloc)

# save tabnet model
saving_path_name = "./tabnet_model_heloc"
saved_filepath = model_heloc.save_model(saving_path_name)

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.971   | train_accuracy: 0.54899 | valid_accuracy: 0.57432 |  0:00:00s
epoch 1  | loss: 0.81065 | train_accuracy: 0.56532 | valid_accuracy: 0.57658 |  0:00:01s
epoch 2  | loss: 0.76538 | train_accuracy: 0.6098  | valid_accuracy: 0.5991  |  0:00:02s
epoch 3  | loss: 0.72161 | train_accuracy: 0.63457 | valid_accuracy: 0.63964 |  0:00:03s
epoch 4  | loss: 0.70833 | train_accuracy: 0.64471 | valid_accuracy: 0.63514 |  0:00:04s
epoch 5  | loss: 0.67887 | train_accuracy: 0.6464  | valid_accuracy: 0.62387 |  0:00:05s
epoch 6  | loss: 0.68401 | train_accuracy: 0.65878 | valid_accuracy: 0.64414 |  0:00:06s
epoch 7  | loss: 0.66425 | train_accuracy: 0.66104 | valid_accuracy: 0.64865 |  0:00:07s
epoch 8  | loss: 0.66057 | train_accuracy: 0.65766 | valid_accuracy: 0.6509  |  0:00:08s
epoch 9  | loss: 0.65499 | train_accuracy: 0.6661  | valid_accuracy: 0.66216 |  0:00:09s
epoch 10 | loss: 0.66365 | train_accuracy: 0.66723 | valid_accuracy: 0.65315 |  0:00:09s
epoch 11 | loss: 0.64

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.96368 | train_accuracy: 0.51633 | valid_accuracy: 0.4955  |  0:00:00s
epoch 1  | loss: 0.82052 | train_accuracy: 0.56813 | valid_accuracy: 0.58333 |  0:00:01s
epoch 2  | loss: 0.74492 | train_accuracy: 0.61486 | valid_accuracy: 0.62838 |  0:00:02s
epoch 3  | loss: 0.71466 | train_accuracy: 0.63964 | valid_accuracy: 0.61486 |  0:00:03s
epoch 4  | loss: 0.69147 | train_accuracy: 0.64077 | valid_accuracy: 0.61261 |  0:00:04s
epoch 5  | loss: 0.67569 | train_accuracy: 0.65653 | valid_accuracy: 0.61712 |  0:00:05s
epoch 6  | loss: 0.65684 | train_accuracy: 0.65991 | valid_accuracy: 0.64414 |  0:00:06s
epoch 7  | loss: 0.6636  | train_accuracy: 0.66667 | valid_accuracy: 0.65315 |  0:00:07s
epoch 8  | loss: 0.6366  | train_accuracy: 0.67005 | valid_accuracy: 0.65541 |  0:00:08s
epoch 9  | loss: 0.65327 | train_accuracy: 0.67962 | valid_accuracy: 0.65541 |  0:00:09s
epoch 10 | loss: 0.63797 | train_accuracy: 0.67793 | valid_accuracy: 0.65991 |  0:00:10s
epoch 11 | loss: 0.62

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 0.92682 | train_accuracy: 0.5366  | valid_accuracy: 0.53604 |  0:00:00s
epoch 1  | loss: 0.83566 | train_accuracy: 0.56588 | valid_accuracy: 0.54279 |  0:00:01s
epoch 2  | loss: 0.72163 | train_accuracy: 0.61543 | valid_accuracy: 0.58108 |  0:00:02s
epoch 3  | loss: 0.7324  | train_accuracy: 0.64752 | valid_accuracy: 0.61712 |  0:00:03s
epoch 4  | loss: 0.68891 | train_accuracy: 0.65146 | valid_accuracy: 0.64414 |  0:00:04s
epoch 5  | loss: 0.69031 | train_accuracy: 0.66216 | valid_accuracy: 0.64189 |  0:00:05s
epoch 6  | loss: 0.67634 | train_accuracy: 0.66667 | valid_accuracy: 0.62613 |  0:00:06s
epoch 7  | loss: 0.65824 | train_accuracy: 0.6661  | valid_accuracy: 0.63288 |  0:00:07s
epoch 8  | loss: 0.64853 | train_accuracy: 0.6723  | valid_accuracy: 0.61036 |  0:00:08s
epoch 9  | loss: 0.63562 | train_accuracy: 0.67624 | valid_accuracy: 0.63739 |  0:00:09s
epoch 10 | loss: 0.62354 | train_accuracy: 0.67849 | valid_accuracy: 0.64189 |  0:00:10s
epoch 11 | loss: 0.61

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.95767 | train_accuracy: 0.52365 | valid_accuracy: 0.49099 |  0:00:00s
epoch 1  | loss: 0.79685 | train_accuracy: 0.54955 | valid_accuracy: 0.55405 |  0:00:01s
epoch 2  | loss: 0.75958 | train_accuracy: 0.61543 | valid_accuracy: 0.60811 |  0:00:02s
epoch 3  | loss: 0.70544 | train_accuracy: 0.63626 | valid_accuracy: 0.62838 |  0:00:03s
epoch 4  | loss: 0.68738 | train_accuracy: 0.6402  | valid_accuracy: 0.61486 |  0:00:04s
epoch 5  | loss: 0.67496 | train_accuracy: 0.64358 | valid_accuracy: 0.61486 |  0:00:05s
epoch 6  | loss: 0.66763 | train_accuracy: 0.65259 | valid_accuracy: 0.62613 |  0:00:06s
epoch 7  | loss: 0.66721 | train_accuracy: 0.65259 | valid_accuracy: 0.62387 |  0:00:07s
epoch 8  | loss: 0.65911 | train_accuracy: 0.65541 | valid_accuracy: 0.6464  |  0:00:08s
epoch 9  | loss: 0.66316 | train_accuracy: 0.66385 | valid_accuracy: 0.64865 |  0:00:09s
epoch 10 | loss: 0.64286 | train_accuracy: 0.67117 | valid_accuracy: 0.63739 |  0:00:10s
epoch 11 | loss: 0.63

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Total Accuracy:  0.7121621621621622
Successfully saved model at ./tabnet_model_heloc.zip


## HIGGS

---

In [59]:
model_higgs = run_tabnet(X_higgs, y_higgs)

# save tabnet model
saving_path_name = "./tabnet_model_higgs"
saved_filepath = model_higgs.save_model(saving_path_name)

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.63092 | train_accuracy: 0.74628 | valid_accuracy: 0.74202 |  0:00:19s
epoch 1  | loss: 0.50916 | train_accuracy: 0.79643 | valid_accuracy: 0.79698 |  0:00:38s
epoch 2  | loss: 0.4532  | train_accuracy: 0.81271 | valid_accuracy: 0.81154 |  0:00:57s
epoch 3  | loss: 0.43311 | train_accuracy: 0.82164 | valid_accuracy: 0.8219  |  0:01:16s
epoch 4  | loss: 0.42596 | train_accuracy: 0.82488 | valid_accuracy: 0.82421 |  0:01:37s
epoch 5  | loss: 0.42206 | train_accuracy: 0.82486 | valid_accuracy: 0.82243 |  0:01:56s
epoch 6  | loss: 0.41766 | train_accuracy: 0.82863 | valid_accuracy: 0.82756 |  0:02:16s
epoch 7  | loss: 0.41081 | train_accuracy: 0.82559 | valid_accuracy: 0.82525 |  0:02:35s
epoch 8  | loss: 0.40983 | train_accuracy: 0.82651 | valid_accuracy: 0.82515 |  0:02:55s
epoch 9  | loss: 0.40519 | train_accuracy: 0.82677 | valid_accuracy: 0.82285 |  0:03:14s
epoch 10 | loss: 0.40438 | train_accuracy: 0.83637 | valid_accuracy: 0.83541 |  0:03:33s
epoch 11 | loss: 0.39

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.63819 | train_accuracy: 0.70699 | valid_accuracy: 0.70286 |  0:00:19s
epoch 1  | loss: 0.53097 | train_accuracy: 0.80486 | valid_accuracy: 0.80013 |  0:00:38s
epoch 2  | loss: 0.45229 | train_accuracy: 0.81886 | valid_accuracy: 0.81541 |  0:00:58s
epoch 3  | loss: 0.43191 | train_accuracy: 0.82415 | valid_accuracy: 0.82201 |  0:01:19s
epoch 4  | loss: 0.42542 | train_accuracy: 0.82541 | valid_accuracy: 0.8196  |  0:01:40s
epoch 5  | loss: 0.41569 | train_accuracy: 0.82831 | valid_accuracy: 0.82316 |  0:02:02s
epoch 6  | loss: 0.41294 | train_accuracy: 0.83297 | valid_accuracy: 0.8285  |  0:02:22s
epoch 7  | loss: 0.40707 | train_accuracy: 0.82407 | valid_accuracy: 0.81751 |  0:02:42s
epoch 8  | loss: 0.4033  | train_accuracy: 0.82836 | valid_accuracy: 0.82326 |  0:03:02s
epoch 9  | loss: 0.40068 | train_accuracy: 0.83274 | valid_accuracy: 0.82536 |  0:03:21s
epoch 10 | loss: 0.39838 | train_accuracy: 0.83198 | valid_accuracy: 0.82525 |  0:03:41s
epoch 11 | loss: 0.39

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 0.63476 | train_accuracy: 0.73243 | valid_accuracy: 0.73249 |  0:00:19s
epoch 1  | loss: 0.52388 | train_accuracy: 0.79936 | valid_accuracy: 0.80149 |  0:00:39s
epoch 2  | loss: 0.46456 | train_accuracy: 0.81253 | valid_accuracy: 0.81384 |  0:00:59s
epoch 3  | loss: 0.44287 | train_accuracy: 0.82093 | valid_accuracy: 0.82452 |  0:01:18s
epoch 4  | loss: 0.43291 | train_accuracy: 0.82441 | valid_accuracy: 0.82808 |  0:01:38s
epoch 5  | loss: 0.42812 | train_accuracy: 0.82606 | valid_accuracy: 0.83101 |  0:01:58s
epoch 6  | loss: 0.41969 | train_accuracy: 0.82381 | valid_accuracy: 0.82777 |  0:02:18s
epoch 7  | loss: 0.41351 | train_accuracy: 0.8309  | valid_accuracy: 0.83279 |  0:02:38s
epoch 8  | loss: 0.40808 | train_accuracy: 0.8314  | valid_accuracy: 0.83143 |  0:02:58s
epoch 9  | loss: 0.40859 | train_accuracy: 0.83145 | valid_accuracy: 0.83394 |  0:03:18s
epoch 10 | loss: 0.40314 | train_accuracy: 0.83399 | valid_accuracy: 0.83719 |  0:03:38s
epoch 11 | loss: 0.39

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.63632 | train_accuracy: 0.7267  | valid_accuracy: 0.72304 |  0:00:19s
epoch 1  | loss: 0.53217 | train_accuracy: 0.79018 | valid_accuracy: 0.78901 |  0:00:38s
epoch 2  | loss: 0.46675 | train_accuracy: 0.80793 | valid_accuracy: 0.80461 |  0:00:58s
epoch 3  | loss: 0.43922 | train_accuracy: 0.81094 | valid_accuracy: 0.80607 |  0:01:17s
epoch 4  | loss: 0.42713 | train_accuracy: 0.82175 | valid_accuracy: 0.81696 |  0:01:37s
epoch 5  | loss: 0.42171 | train_accuracy: 0.82175 | valid_accuracy: 0.81592 |  0:01:56s
epoch 6  | loss: 0.41526 | train_accuracy: 0.82703 | valid_accuracy: 0.81791 |  0:02:16s
epoch 7  | loss: 0.41282 | train_accuracy: 0.81923 | valid_accuracy: 0.8155  |  0:02:35s
epoch 8  | loss: 0.40865 | train_accuracy: 0.82667 | valid_accuracy: 0.82304 |  0:02:55s
epoch 9  | loss: 0.40589 | train_accuracy: 0.83583 | valid_accuracy: 0.82995 |  0:03:14s
epoch 10 | loss: 0.40477 | train_accuracy: 0.82735 | valid_accuracy: 0.82346 |  0:03:34s
epoch 11 | loss: 0.40

C:\Users\johnp\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Total Accuracy:  0.8376471266680225
Successfully saved model at ./tabnet_model_higgs.zip


---

## Calculate the final predictions for submission

In [33]:
#Load the save TabNet model for the CoverType dataset
model_cov = TabNetClassifier()
model_cov.load_model("./tabnet_model_cov.zip")

In [34]:
#Load the save TabNet model for the HELOC dataset
model_heloc = TabNetClassifier()
model_heloc.load_model("./tabnet_model_heloc.zip")

In [35]:
#Load the save TabNet model for the HIGGS dataset
model_higgs = TabNetClassifier()
model_higgs.load_model("./tabnet_model_higgs.zip")

In [55]:
final_preds, final_preds_cov, final_preds_heloc, final_preds_higgs = get_final_predictions(model_cov, model_heloc, model_higgs, False)

In [41]:
le = LabelEncoder()

final_preds_heloc = le.fit_transform(final_preds_heloc)

In [43]:
le2 = LabelEncoder()

final_preds_higgs = le.fit_transform(final_preds_higgs)

In [45]:
final_preds = np.concatenate((final_preds_cov, final_preds_heloc, final_preds_higgs))

---

### Export the final predictions into zip files

In [59]:
sol_output = pd.DataFrame(final_preds, index=range(1, 1 + len(final_preds)))
sol_output.rename(columns={0: "Prediction"}, inplace=True)
sol_output.index.name = 'ID'
compression_opts = dict(method='zip',
                        archive_name='predictions_x.csv')  
sol_output.to_csv('out_x.zip', index=True,
          compression=compression_opts)

In [48]:
sol_output = pd.DataFrame(final_preds_cov, index=range(1, 1 + len(final_preds_cov)))
sol_output.rename(columns={0: "Prediction"}, inplace=True)
sol_output.index.name = 'ID'
compression_opts = dict(method='zip',
                        archive_name='predictions_tab_cov_2.csv')  
sol_output.to_csv('out_tab_cov_2.zip', index=True,
          compression=compression_opts)

In [61]:
sol_output = pd.DataFrame(final_preds_heloc, index=range(3501, 3501 + len(final_preds_heloc)))
sol_output.rename(columns={0: "Prediction"}, inplace=True)
sol_output.index.name = 'ID'
compression_opts = dict(method='zip',
                        archive_name='predictions_x_heloc.csv')  
sol_output.to_csv('out_x_heloc.zip', index=True,
          compression=compression_opts)

In [62]:
sol_output = pd.DataFrame(final_preds_higgs, index=range(4547, 4547 + len(final_preds_higgs)))
sol_output.rename(columns={0: "Prediction"}, inplace=True)
sol_output.index.name = 'ID'
compression_opts = dict(method='zip',
                        archive_name='predictions_x_higgs.csv')  
sol_output.to_csv('out_x_higgs.zip', index=True,
          compression=compression_opts)